# Excitable System Model

This notebook implements a 2-dimensional ODE model of an excitable system representing immune response dynamics.

## Model Equations
$$\frac{dx}{dt} = x - \frac{x^3}{3} - y + I(t)$$
$$\frac{dy}{dt} = \epsilon (x + \beta - \gamma y)$$

## Parameters
- $\epsilon = 0.08$
- $\gamma = 0.5$
- $\beta = 0.7$
- $I(t)$ is a box car function: $I(t) = 0.5$ for $t \in [10, 10+T]$ and $0$ otherwise.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
# Parameters
epsilon = 0.08
gamma = 0.5
beta = 0.7
I_mag = 0.5
t_start_stim = 10.0

def input_signal(t, T):
    if t_start_stim <= t <= t_start_stim + T:
        return I_mag
    return 0.0

def excitable_model(t, z, T):
    x, y = z
    I = input_signal(t, T)
    dxdt = x - (x**3)/3 - y + I
    dydt = epsilon * (x + beta - gamma * y)
    return [dxdt, dydt]

In [ ]:
# Initial conditions (Fixed point calculation for I=0)
# x^3 + 3x + 4.2 = 0
coeffs = [1, 0, 3, 4.2]
roots = np.roots(coeffs)
real_roots = roots[np.isreal(roots)].real
x0_val = real_roots[0]
y0_val = x0_val - (x0_val**3)/3

y0 = [x0_val, y0_val]
print(f"Initial conditions: x0={x0_val:.4f}, y0={y0_val:.4f}")

In [ ]:
def compute_fwhm(t, y):
    """
    Compute Full Width at Half Maximum (FWHM) for a signal y(t).
    Here we consider the peak above the baseline.
    """
    baseline = np.min(y)
    peak = np.max(y)
    half_max = (peak - baseline) / 2 + baseline
    
    # Find indices where signal is above half max
    # This is a basic implementation
    above_half = np.where(y >= half_max)[0]
    if len(above_half) > 0:
        t_start = t[above_half[0]]
        t_end = t[above_half[-1]]
        fwhm = t_end - t_start
        return fwhm, t_start, t_end
    return 0.0, None, None

In [ ]:
def run_simulation(T):
    t_span = [0, 100]
    sol = solve_ivp(excitable_model, t_span, y0, args=(T,), max_step=0.1)
    return sol

# Run simulations for different durations T
T_short = 0.5
T_long = 3.0

sol_short = run_simulation(T_short)
sol_long = run_simulation(T_long)

# Calculate FWHM for the firing case
fwhm_long, _, _ = compute_fwhm(sol_long.t, sol_long.y[0])
print(f"FWHM for T={T_long}: {fwhm_long:.2f}")

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(sol_short.t, sol_short.y[0], label=f'x (T={T_short})', linestyle='--')
plt.plot(sol_long.t, sol_long.y[0], label=f'x (T={T_long})', linewidth=2)
plt.axhline(y=x0_val, color='k', linestyle=':', label='Baseline')
plt.xlabel('Time')
plt.ylabel('x (Variable)')
plt.title('Excitable System Dynamics')
plt.legend()
plt.grid(True)
plt.show()